# Materials data science: data retrieval and filtering – Exercises

## Exercise 1: Load and examine the `elastic_tensor_2015` dataset

Matminer includes a dataset called `elastic_tensor_2015`. It contains a set of computed elastic properties of materials sourced from the paper:

> "Charting the complete elastic properties of inorganic crystalline compounds", M. de Jong et al., Sci. Data. 2 (2015) 150009."

Load this dataset using the `load_dataset()` function and determine:
- the number of entries it contains (tip: pandas `DataFrame` objects have a `describe()` function)
- the largest value of bulk modulus in the dataset (bulk modulus is given in the `K_VRH` column)

In [1]:
from matminer.datasets import load_dataset

df = load_dataset("elastic_tensor_2015")

df.describe()

,nsites,space_group,volume,elastic_anisotropy,G_Reuss,G_VRH,G_Voigt,K_Reuss,K_VRH,K_Voigt,poisson_ratio,kpoint_density
count,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000
mean,12.425910,163.403895,207.177098,2.145013,64.050568,67.543145,71.035720,135.171392,136.259661,137.347932,0.287401,7536.833192
std,11.817997,65.040733,192.355747,19.140097,44.696380,44.579408,45.388731,73.582579,72.886978,72.922887,0.062177,3446.890979
min,2.000000,4.000000,15.850527,0.000005,1.870270,2.722175,3.574080,4.714976,6.476135,6.476138,0.042582,1000.000000
25%,5.000000,124.000000,83.944059,0.145030,30.244413,34.117959,37.270657,74.960699,76.435350,76.520333,0.249159,7000.000000
50%,10.000000,193.000000,168.920404,0.355287,56.263878,59.735163,62.635382,129.984790,130.382766,131.849056,0.290198,7000.000000
75%,16.000000,221.000000,261.420345,0.923117,86.979486,91.332142,95.785011,189.195104,189.574194,190.912352,0.328808,7000.000000
max,152.000000,229.000000,2398.906164,397.297866,520.845926,522.921225,524.996524,435.658754,435.661487,435.664220,0.467523,45000.000000


## Exercise 2: Filter the dataset based on the number of sites

You are constructing a machine learning model for elastic constants that is only designed to be employed on structures containing a small number of atomic sites. You should filter the dataset to only include entries where `nsites` is less than 20 and determine:
- the number of entries in the filtered dataset
- the mean number of sites across all entries in your filtered dataset *(hint: `pandas` Dataframes have a `.mean()` method you can use on a column to get the mean of that column)*


In [2]:
from matminer.datasets import load_dataset

df = load_dataset("elastic_tensor_2015")

# complete exercise below

mask = df["nsites"] < 20
filtered_df = df[mask]

n_filtered = len(filtered_df)
mean_n_sites = filtered_df["nsites"].mean()

print("# entries in filtered dataset: {}".format(n_filtered))
print("mean # sites: {}".format(mean_n_sites))

# entries in filtered dataset: 975
mean # sites: 8.554871794871795


## Exercise 3: Remove columns unnecessary for machine learning

The elastic tensor dataset contains many columns that are not particular relevant for machine learning. You should filter the dataset, so that it only contains the `formula`, `structure`, and `K_VRH` (bulk modulus) columns.

*Tip: there are two ways to do this exercise: (1) using indexing with a list, and (2)  using the pandas `DataFrame.drop()` method that can be used for dropping both rows and columns. Make sure you set the `axis` argument correctly.*

In [4]:
from matminer.datasets import load_dataset

df = load_dataset("elastic_tensor_2015")

# complete exercise below

df[["formula", "structure", "K_VRH"]]

# alternatively, you can do
# df.drop(['material_id', 'nsites', 'space_group', 'volume',
#          'elastic_anisotropy', 'G_Reuss', 'G_VRH', 'G_Voigt',
#          'K_Reuss', 'K_Voigt', 'poisson_ratio', 'compliance_tensor',
#          'elastic_tensor', 'elastic_tensor_original', 'cif', 'kpoint_density',
#          'poscar'],
#        axis=1)

,formula,structure,K_VRH
0,Nb4CoSi,"[[0.94814328 2.07280467 2.5112 ] Nb, [5.273...",194.268884
1,Al(CoSi)2,"[[0. 0. 0.] Al, [1.96639263 1.13529553 0.75278...",175.449907
2,SiOs,"[[1.480346 1.480346 1.480346] Si, [0. 0. 0.] Os]",295.077545
3,Ga,"[[0. 1.09045794 0.84078375] Ga, [0. ...",49.130670
4,SiRu2,"[[1.0094265 4.24771709 2.9955487 ] Si, [3.028...",256.768081
...,...,...,...
1176,Ti2CdC,"[[1.55171489 0.89588144 8.41159136] Ti, [-1.88...",111.788114
1177,ScSi,"[[0. 8.53417539 0.91740975] Sc, [0. ...",101.326807
1178,YSi,"[[0. 9.084549 0.960921] Y, [0. 1.4...",89.418161
1179,Al2Cu,"[[5.11035838 2.07486738 0. ] Al, [3.996...",99.384565


## Advanced exercise: calculate Young's modulus

Young's modulus, $E$, is given by:

$$
E = \frac{9KG}{G+3K},
$$

where $K$ is the bulk modulus (column `K_VRH`), and $G$ is the shear modulus (column `G_VRH`).

Calculate Young's modulus for all entries in the dataset and store them in a new column called `E_VRH`. What is the average Young modulus over the entire dataset?

In [5]:
from matminer.datasets import load_dataset

df = load_dataset("elastic_tensor_2015")

# complete exercise below

df["E_VRH"] = (9 * df["K_VRH"] * df["G_VRH"]) / (df["G_VRH"] + 3 * df["K_VRH"])

df

,material_id,formula,nsites,space_group,volume,structure,elastic_anisotropy,G_Reuss,G_VRH,G_Voigt,...,K_VRH,K_Voigt,poisson_ratio,compliance_tensor,elastic_tensor,elastic_tensor_original,cif,kpoint_density,poscar,E_VRH
0,mp-10003,Nb4CoSi,12,124,194.419802,"[[0.94814328 2.07280467 2.5112 ] Nb, [5.273...",0.030688,96.844535,97.141604,97.438674,...,194.268884,194.270146,0.285701,"[[0.004385293093993, -0.0016070693558990002, -...","[[311.33514638650246, 144.45092552856926, 126....","[[311.33514638650246, 144.45092552856926, 126....",#\#CIF1.1\n###################################...,7000,Nb8 Co2 Si2\n1.0\n6.221780 0.000000 0.000000\n...,249.790066
1,mp-10010,Al(CoSi)2,5,164,61.987320,"[[0. 0. 0.] Al, [1.96639263 1.13529553 0.75278...",0.266910,93.939650,96.252006,98.564362,...,175.449907,177.252050,0.268105,"[[0.0037715428949660003, -0.000844229828709, -...","[[306.93357350984974, 88.02634955100905, 105.6...","[[306.93357350984974, 88.02634955100905, 105.6...",#\#CIF1.1\n###################################...,7000,Al1 Co2 Si2\n1.0\n3.932782 0.000000 0.000000\n...,244.115367
2,mp-10015,SiOs,2,221,25.952539,"[[1.480346 1.480346 1.480346] Si, [0. 0. 0.] Os]",0.756489,120.962289,130.112955,139.263621,...,295.077545,295.077545,0.307780,"[[0.0019959391925840004, -0.000433146670736000...","[[569.5291276937579, 157.8517489654999, 157.85...","[[569.5291276937579, 157.8517489654999, 157.85...",#\#CIF1.1\n###################################...,7000,Si1 Os1\n1.0\n2.960692 0.000000 0.000000\n0.00...,340.318316
3,mp-10021,Ga,4,63,76.721433,"[[0. 1.09045794 0.84078375] Ga, [0. ...",2.376805,12.205989,15.101901,17.997812,...,49.130670,49.235377,0.360593,"[[0.021647143908635, -0.005207263618160001, -0...","[[69.28798774976904, 34.7875015216915, 37.3877...","[[70.13259066665267, 40.60474945058445, 37.387...",#\#CIF1.1\n###################################...,7000,Ga4\n1.0\n2.803229 0.000000 0.000000\n0.000000...,41.095069
4,mp-10025,SiRu2,12,62,160.300999,"[[1.0094265 4.24771709 2.9955487 ] Si, [3.028...",0.196930,100.110773,101.947798,103.784823,...,256.768081,258.480904,0.324682,"[[0.00410214297725, -0.001272204332729, -0.001...","[[349.3767766177825, 186.67131003104407, 176.4...","[[407.4791016459293, 176.4759188081947, 213.83...",#\#CIF1.1\n###################################...,7000,Si4 Ru8\n1.0\n4.037706 0.000000 0.000000\n0.00...,270.096776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,mp-9961,Ti2CdC,8,194,121.520152,"[[1.55171489 0.89588144 8.41159136] Ti, [-1.88...",2.229538,43.697362,53.329789,62.962216,...,111.788114,113.178476,0.294196,"[[0.0044568832753350005, -0.001136458158149, -...","[[250.07723993082595, 71.61036453717465, 44.31...","[[250.07723993082595, 71.61036453717465, 44.31...",#\#CIF1.1\n###################################...,7000,Ti4 Cd2 C2\n1.0\n3.103427 0.000000 0.000000\n-...,138.038433
1177,mp-9969,ScSi,8,63,145.392095,"[[0. 8.53417539 0.91740975] Sc, [0. ...",0.635889,66.067578,70.253182,74.438787,...,101.326807,101.446005,0.218412,"[[0.007188192080993001, -0.000983458215036, -0...","[[172.01931871370775, 43.38593006033102, 76.22...","[[172.01931871370775, 43.38593006033102, 76.22...",#\#CIF1.1\n###################################...,7000,Sc4 Si4\n1.0\n3.996960 0.000000 0.000000\n0.00...,171.194610
1178,mp-9972,YSi,8,63,174.123748,"[[0. 9.084549 0.960921] Y, [0. 1.4...",0.454691,62.326766,65.160458,67.994149,...,89.418161,89.419968,0.206850,"[[0.007749056578454001, -0.001293580144074, -0...","[[159.44792354820547, 42.19645554837514, 66.58...","[[159.44792354820547, 42.19645554837514, 66.58...",#\#CIF1.1\n###################################...,7000,Y4 Si4\n1.0\n4.286685 0.000000 0.000000\n0.000...,157.277759
1179,mp-998,Al2Cu,12,140,179.147816,"[[5.11035838 2.07486738 0. ] Al, [3.996...",0.735449,37.457129,40.211802,42.966475,...,99.384565,99.386015,0.321738,"[[0.007456839159427001, -0.002592061077082, -0...","[[170.12510286354689, 70.6116049091726, 57.026...","[[170.12510286